In [76]:
import tensorflow as tf 
from tensorflow.keras.models import load_model
import joblib 
import pandas as pd 
import numpy as np  

In [77]:
## load the trained model,scaler joblib,onehot 
model=load_model('model.h5')

## load the encoder and scalers 
with open('Onehot_encoder_geo.pkl','rb') as file:
    Onehot_encoder_geo=joblib.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=joblib.load(file)

with open('scaler.pkl','rb') as file:
    scaler=joblib.load(file)

In [78]:
## example inplut data 

input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [79]:
## onehot encode 'Geography'
geo_encoded=Onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=Onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


/opt/anaconda3/envs/My-env/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [80]:
input_df=pd.DataFrame([input_data])

In [81]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [82]:
# encode categorical variables 
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'].values)
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [83]:
## concatination with OHE 
input_df=pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [85]:
input_scaled=scaler.transform(input_df)

In [86]:
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [87]:
## predict Churn 
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 63ms/step


array([[0.01837699]], dtype=float32)

In [88]:
prediction_proba=prediction[0][0]

In [89]:
prediction_proba

0.018376995

In [90]:
if prediction_proba>0.5:
    print("The Customer is likely to churn.")
else:
    print("The Customer is not likely to churn")

The Customer is not likely to churn
